In [ ]:
# Instalar paquetes
!pip install jupyter_plotly_dash
!pip install plotly==4.8.1
!pip install "ipywidgets>=7.2"
!pip install sklearn
!pip install tensorflow
!pip install pandas
!pip install numpy

In [1]:
# Importar paquetes necesarios
import plotly.graph_objects as go
from jupyter_plotly_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
# Importar las funciones de los modelos
from TimeSeriesPrediction import fixed_partitioning_predict
from LinearRegression import linear_regression_predict
from DNN import load_model, dnn_predict, dnn_predict_by_year

# Importar interface de modelos
from Evaluation import Model, TestResult

# Crear modelos

model_dict = {
    'fpts' : Model(fixed_partitioning_predict),
    'fplr' : Model(linear_regression_predict),
    'dnn' : Model(dnn_predict, args = dict(cached_model = load_model('default')))
}

# Crear etiquetas
label_dict = {
    'fpts' : 'Fixed partitioning time series',
    'fplr' : 'Fixed partitioning linear regression',
    'dnn' : 'Redes neuronales', 
    'PrimariasCompletas' : 'primarias públicas y privadas',
    'PrimariasPublicas' : 'primarias públicas',
    'PrimariasPrivadas' : 'primarias privadas'
}

Loaded model  default


In [3]:
# Callbacks reference:
# https://dash.plotly.com/basic-callbacks

fig = go.Figure(data=go.Bar(y=[2, 3, 1]))

def model_management(modelo, prediction_size, set_name) :
    global model_dict
    global label_dict
    model = model_dict[modelo]
    result = model.test_set(set_name, prediction_size)
    result.fig.update_layout(
        title = 'Predicción de matrícula escolar en %s utilizando %s' % (
            label_dict[set_name],
            label_dict[modelo]
        )
    )
    return result.fig
    

app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.H1("Evaluación de modelos para la predicción de matrícula escolar"),
    html.Hr(),
    html.H4("Modelo"),
    dcc.RadioItems(
        id = 'modelos',
        options = [
            {"label": "Fixed partitioning time series", "value": "fpts"},
            {"label": "Fixed partitioning linear regression", "value": "fplr"},
            {"label": "Deep Neural Network", "value": "dnn"}
        ],
        value = "fpts",
        labelStyle = {'display': 'block'}
    ),
    html.H4("Años a predecir"),
    dcc.RadioItems(
        id = 'anios',
        options = [
            {"label": "1", "value": 1},
            {"label": "2", "value": 2},
            {"label": "3", "value": 3},
            {"label": "4", "value": 4},
            {"label": "5", "value": 5}
        ],
        value = 5,
        labelStyle = {'display': 'inline'}
    ),
    html.H4("Conjunto de datos"),
    dcc.RadioItems(
        id = 'conjunto_datos',
        options = [
            {"label": "Primarias públicas y privadas", "value": "PrimariasCompletas"},
            {"label": "Primarias públicas", "value": "PrimariasPublicas"},
            {"label": "Primarias privadas", "value": "PrimariasPrivadas"}
        ],
        value = "PrimariasCompletas",
        labelStyle = {'display': 'block'}
    ),
    html.Button('Evaluar', id='boton_evaluar'),
    html.Hr(),
    html.Div([dcc.Graph(id = "grafica", figure = fig)]),
    
    # CSS
    html.Link(rel = "stylesheet", href = "https://stackpath.bootstrapcdn.com/bootstrap/4.5.0/css/bootstrap.min.css")
])

@app.callback(
    dash.dependencies.Output('grafica', 'figure'),
    [dash.dependencies.Input('modelos', 'value'),
     dash.dependencies.Input('anios', 'value'),
     dash.dependencies.Input('conjunto_datos', 'value'),
     dash.dependencies.Input('boton_evaluar', 'value')
    ])
def mostrar_resultado(modelo, prediction_size, set_name, button_val):
    return model_management(modelo, prediction_size, set_name)

app

In [6]:
print("")